# **DEEP LEARNING**

Name: **EXERCICE**  
Date : 2023  
Author: Aurélien Vannieuwenhuyze  


<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Licence Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under the terms of the <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License.</a>.
<hr/>

## Cas pratique sonar

In [35]:
import numpy as np
import os
import pandas as pnd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [36]:
from tensorflow import keras

keras_model = keras.Sequential()
keras_model.add(keras.layers.Input((60,)))
keras_model.add(keras.layers.Dense(24, activation='softmax', use_bias=True))
keras_model.add(keras.layers.Dense(2, activation='softmax', use_bias=True))

keras_model.compile(loss=keras.losses.binary_crossentropy)

keras_model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 24)                1464      
                                                                 
 dense_17 (Dense)            (None, 2)                 50        
                                                                 
Total params: 1,514
Trainable params: 1,514
Non-trainable params: 0
_________________________________________________________________


In [37]:
data = pnd.read_csv("./sonar.all-data")

input_data = data.sample(frac=1).reset_index(drop=True)

          0       1       2       3       4       5       6       7       8  \
61   0.0086  0.0215  0.0242  0.0445  0.0667  0.0771  0.0499  0.0906  0.1229   
84   0.0365  0.1632  0.1636  0.1421  0.1130  0.1306  0.2112  0.2268  0.2992   
153  0.0117  0.0069  0.0279  0.0583  0.0915  0.1267  0.1577  0.1927  0.2361   
48   0.0119  0.0582  0.0623  0.0600  0.1397  0.1883  0.1422  0.1447  0.0487   
6    0.0519  0.0548  0.0842  0.0319  0.1158  0.0922  0.1027  0.0613  0.1465   
..      ...     ...     ...     ...     ...     ...     ...     ...     ...   
200  0.0335  0.0258  0.0398  0.0570  0.0529  0.1091  0.1709  0.1684  0.1865   
101  0.0587  0.1210  0.1268  0.1498  0.1436  0.0561  0.0832  0.0672  0.1372   
189  0.0156  0.0210  0.0282  0.0596  0.0462  0.0779  0.1365  0.0780  0.1038   
182  0.0096  0.0404  0.0682  0.0688  0.0887  0.0932  0.0955  0.2140  0.2546   
55   0.0152  0.0102  0.0113  0.0263  0.0097  0.0391  0.0857  0.0915  0.0949   

          9  ...      50      51      52      53   

In [38]:
class analyseEpoque(keras.callbacks.Callback):
    def __init__(self, x_test, y_test):
        self.x_test = x_test
        self.y_test = y_test
        
    def on_epoch_end(self, epoch, logs=None):
        print(self.x_test)
        print(self.y_test)
        y_pred = self.model.predict(self.x_test)
        print('Prédiction: {} a epoque: {}'.format(y_pred, epoch))
        print('Attendu: {} a epoque: {}'.format(self.y_test, epoch))

In [39]:
keras_model.fit(X_train, Y_train,  epochs=10,callbacks=[analyseEpoque(X_train,Y_train)])

Epoch 1/10


ValueError: in user code:

    File "/home/kilian/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/kilian/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/kilian/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/home/kilian/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/kilian/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "/home/kilian/.local/lib/python3.10/site-packages/keras/engine/compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_

In [ ]:
predictions = keras_model.predict(X_test)

predictions = [int(pred[1] + pred[0] / 2 > 0.5) for pred in predictions]

comparatif = [(predictions[i], Y_test.iloc[i]) for i in range(len(predictions))]
tx_erreur = sum([comp[0] - comp[1] for comp in comparatif]) / len(comparatif)

print(comparatif, tx_erreur)

4/4 [==============================] - 0s 1ms/step
[(1, 1), (1, 1), (1, 1), (1, 1), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 1), (1, 1), (1, 1), (1, 0), (1, 1), (1, 0), (1, 1), (1, 1), (1, 0), (1, 1), (1, 0), (1, 1), (1, 1), (1, 1), (1, 1), (1, 0), (1, 1), (1, 0), (1, 0), (1, 0), (1, 1), (1, 0), (1, 0), (1, 1), (1, 1), (1, 1), (1, 0), (1, 0), (1, 0), (1, 1), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 1), (1, 0), (1, 1), (1, 0), (1, 0), (1, 1), (1, 0), (1, 1), (1, 1), (1, 1), (1, 1), (1, 0), (1, 0), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 0), (1, 0), (1, 1), (1, 0), (1, 1), (1, 1), (1, 1), (1, 1), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 1), (1, 0), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 0), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 0), (1, 0), (1, 0), (1, 1), (1, 0), (1, 1), (1, 0), (1, 0), (1, 1), (1, 1), (1, 1), (1, 1)] 0.47115384615384615


Pdf de l'étude (1988):  
https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.299.8959&rep=rep1&type=pdf

## Chargement du dataset

http://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+%28Sonar%2C+Mines+vs.+Rocks%29